# PROCESSAMENTO DA TABELA CLIENTES

In [1]:
#instala módulos utilizados
#!pip install chardet
#!pip install openpyxl

In [2]:
from collections import Counter
from datetime import datetime
import chardet
import numpy as np
import os
import pandas as pd
import re

### Definição de dados

In [3]:
#Diretório da pasta com os dados do escritório
pasta_dados = ('./backup_de_dados/')
#Código do cliente que será atualizado
#Esse valor é o campo CodEmpresa nos arquivos
cod_cliente_advbox = 92577

In [4]:
def detectar_codificacao(caminho_arquivo):
    with open(caminho_arquivo, "rb") as f:
        resultado = chardet.detect(f.read())
        return resultado["encoding"]

In [5]:
def carrega_csv(arquivo):
    codificacao = detectar_codificacao(arquivo)
    return pd.read_csv(arquivo, delimiter=';',encoding=codificacao)

### Carregamento das tabelas

In [6]:
padrao_backup = re.compile(
    r"v_(.*?)_CodEmpresa_" + 
    re.escape(str(cod_cliente_advbox)) + 
    r"\.csv")

In [7]:
tabelas = {}
for arquivo in os.listdir(pasta_dados):
    if padrao_backup.match(arquivo):
        tabelas[padrao_backup.match(arquivo).group(1)]= carrega_csv(pasta_dados+arquivo)

In [8]:
len(tabelas)

32

In [9]:
df = tabelas['clientes']
df.replace({np.nan: None},inplace=True)
df.head()

,codigo,codempresa,codigo_administrativo,codigo_unit,razao_social,nome_fantasia,responsavel,cnpj,contato_nome,contato_ddd1,...,campo_livre2,cod_campo_livre2,cod_terceiro_categoria,numero_pasta,migracao_parceiro,cod_cliente_estado_civil,cod_usuario,cod_estado_brasil,migracao3,cod_escolaridade
0,106207729,92577,None,0,Fulano(a) 1,None,None,None,None,None,...,None,None,None,1367.0,None,None,None,None,None,None
1,106206605,92577,None,0,Fulano(a) 2,None,None,None,None,None,...,None,None,None,1366.0,None,None,None,None,None,None
2,8897738,92577,None,0,Fulano(a) 3,None,None,None,None,None,...,None,7461.0,None,341.0,None,None,None,None,None,None
3,5251301,92577,None,0,Fulano(a) 4,None,None,None,None,None,...,None,6280.0,None,160.0,None,None,None,None,None,None
4,13203419,92577,None,0,Fulano(a) 5,None,None,None,Jana,None,...,None,6280.0,None,560.0,None,None,None,None,None,None


In [10]:
print(f"A tabela clientes possui {df.shape[0]} linhas e {df.shape[1]} colunas.")

A tabela clientes possui 1828 linhas e 83 colunas.


Criando os dataframe dos clientes

In [11]:
df_clientes=pd.DataFrame(columns=[
    'nomeCliente',
    'cpfcnpj',
    'rg',
    'nacionalidade',
    'dataDeNascimento',
    'estadoCivil',
    'profissao',
    'sexo',
    'celular',
    'telefone',
    'email',
    'pais',
    'estado',
    'cidade',
    'bairro',
    'endereco',
    'cep',
    'pispasep',
    'ctps',
    'cid',
    'nomemae',
    'origem',
    'anotacoesGerais'])

In [12]:
df_clientes

,nomeCliente,cpfcnpj,rg,nacionalidade,dataDeNascimento,estadoCivil,profissao,sexo,celular,telefone,...,cidade,bairro,endereco,cep,pispasep,ctps,cid,nomemae,origem,anotacoesGerais


### Verificando entradas duplicadas

In [13]:
print(f"Número de entradas duplicadas na tabela: {df.duplicated(subset='codigo').sum()}.")

Número de entradas duplicadas na tabela: 21.


In [14]:
df.drop_duplicates(subset='codigo', keep='first', inplace=True, ignore_index=True)

### Verificando nomes duplicados

In [15]:
print(f"Número de linhas restantes: {df.shape[0]}.")

Número de linhas restantes: 1807.


In [16]:
print(f"Número de nomes sem duplicados restantes: {len(df['razao_social'].unique())}.")

Número de nomes sem duplicados restantes: 1807.


Não restam nomes duplicados.

## Verificando campo CPF

In [17]:
resultados = 0
for index, row in df.iterrows():
    valor_col1 = row['cnpj']
    valor_col2 = row['cpf']
    if pd.isna(valor_col1) and pd.isna(valor_col2):
        pass
    elif pd.isna(valor_col1) or pd.isna(valor_col2):
        pass
    else:
        resultados + 1
print(f"Número de linhas com o CPF ou o CNPJ preenchidos ao mesmo tempo: {resultados}.")

Número de linhas com o CPF ou o CNPJ preenchidos ao mesmo tempo: 0.


In [18]:
def combinar_cpf_cnpj(row):
    if row['cpf'] is not None:
        return row['cpf']
    elif row['cnpj'] is not None:
        return row['cnpj']
    return None
df['cpfcnpjcombinado'] = df.apply(combinar_cpf_cnpj, axis=1)

In [19]:
def processar_cpf_cnpj(row):
    coluna1 = ''.join(filter(str.isdigit, str(row['cpf_cnpj'])))
    coluna2 = ''.join(filter(str.isdigit, str(row['cpfcnpjcombinado'])))
    if not coluna1:
        return coluna2, None
    if not coluna2:
        return coluna1, None
    if row['cliente'] == 'FI':
        coluna1.zfill(11)
        coluna2.zfill(11)
    else:
        coluna1.zfill(14)
        coluna2.zfill(14)
    if coluna1 == coluna2:
        return coluna1, None
    return coluna1, coluna2

In [20]:
df[['cpfCnpjfinal', 'valorCpfCNPJDivergente']] = df.apply(processar_cpf_cnpj, axis=1, result_type='expand')
print(f"Número de linhas que possuiam valores de CPF ou CNPJ divergentes: {tabelas['clientes']['valorCpfCNPJDivergente'].sum()}.")

Número de linhas que possuiam valores de CPF ou CNPJ divergentes: 0.


In [21]:
def validar_documento(documento):
    documento = ''.join(filter(str.isdigit, str(documento)))
    if len(documento) < 11:
        documento = documento.zfill(11)
        tipo = 'cpf'
    elif len(documento) == 11:
        tipo = 'cpf'
    elif len(documento) <= 14:
        documento = documento.zfill(14)
        tipo = 'cnpj'
    elif len(documento) > 14:
        return None, documento
    if tipo == 'cpf':
        if len(documento) != 11 or documento == documento[0] * 11:
            return None, documento
        soma1 = sum(int(documento[i]) * (10 - i) for i in range(9))
        resto1 = soma1 % 11
        digito1 = 0 if resto1 < 2 else 11 - resto1
        soma2 = sum(int(documento[i]) * (11 - i) for i in range(10))
        resto2 = soma2 % 11
        digito2 = 0 if resto2 < 2 else 11 - resto2
        if documento[-2:] == f'{digito1}{digito2}':
            return documento[:3] + documento[3:6] + documento[6:9] + documento[9:], None
        else:
            return None, documento
    if tipo == 'cnpj':
        if len(documento) != 14 or documento == documento[0] * 14:
            return None, documento
        pesos1 = [5, 4, 3, 2, 9, 8, 7, 6, 5, 4, 3, 2]
        soma1 = sum(int(documento[i]) * pesos1[i] for i in range(12))
        resto1 = soma1 % 11
        digito1 = 0 if resto1 < 2 else 11 - resto1
        pesos2 = [6, 5, 4, 3, 2, 9, 8, 7, 6, 5, 4, 3, 2]
        soma2 = sum(int(documento[i]) * pesos2[i] for i in range(13))
        resto2 = soma2 % 11
        digito2 = 0 if resto2 < 2 else 11 - resto2
        if documento[-2:] == f'{digito1}{digito2}':
            return documento[:2] + documento[2:5] + documento[5:8] + documento[8:12] + documento[12:], None
        else:
            return None, documento

In [22]:
df[['CPFCNPJ', 'CPFCNPJ_invalido']] = df['cpfCnpjfinal'].apply(lambda x: pd.Series(validar_documento(x)))

## Campo RG

Campo RG é igual ao de entrada

## Campo Nacionalidade

In [23]:
print(df['nacionalidade'].unique())

['Brasileiro' 'Brasileira' 'brasileira' None 'Brasileria' 'brasileiro'
 'Brasileiira' 'Brasielira' 'Brsaileira' 'Brasilira' 'Brasiliero'
 'Brsileiro' 'Brasiliera' 'Brasilieira' 'Braisleira' 'BRASILEIRA'
 'BRASILEIRO' 'braileira']


Todas as nacionalidades são brasileiras, apesar dos erros de digitação

In [24]:
df['nacionalidade'] = df['nacionalidade'].apply(lambda x: 'Brasileira' if x is not None else x)

## Campo data de nascimento

In [25]:
#Converte as datas de nascimento para o formato solicitado
df['nascimento'] = pd.to_datetime(df['nascimento'], format='%d/%m/%Y %H:%M')
df['nascimento']= df['nascimento'].dt.strftime('%d/%m/%Y')

### Campo estado civil
Os identificadores estão na tabela cliente_estado_civil, porém foi solicitado para deixar como está.

### Campo profissão
Será mantida como está

### Campo Sexo
Não há informações no cadastro de clientes que permitam inferir o sexo. Será deixado vazio.

### Campo Celular
Analisando os dados, o campo telefone2 é o que possui os números de celular

In [26]:
def contar_algarismos_por_entrada(dataframe, coluna):
    contagem_algarismos = []
    for entrada in dataframe[coluna]:
        apenas_numeros = ''.join(filter(str.isdigit, str(entrada)))
        if apenas_numeros:
            contagem_algarismos.append(len(apenas_numeros))
    contador = Counter(contagem_algarismos)
    resultado = ",".join(f"{k}:{v}" for k, v in sorted(contador.items()))
    return resultado

In [27]:
print(contar_algarismos_por_entrada(df, 'telefone2'))

10:1635


Todos os resultados possuem apenas 10 dígitos.

In [28]:
df['celular'] = df['telefone2'].apply(
    lambda x: None if not x or not (numeros := ''.join(filter(str.isdigit, str(x)))) 
              else f"({numeros[:2]}) {numeros[2:6]}-{numeros[6:]}")

### Campo Telefone
telefone1 ou telefone3, o que estiver preenchido

In [29]:
print(contar_algarismos_por_entrada(df, 'telefone1'))

10:1007


In [30]:
print(contar_algarismos_por_entrada(df, 'telefone3'))

10:630


Ambos os campos de telefone possuem 10 algarismos, então não há necessidade de realizar formatações adicionais.

In [31]:
df['telefone'] = df.apply(
    lambda row: (
        f"({(numeros := ''.join(filter(str.isdigit, str(row['telefone1']))))[:2]}) {numeros[2:6]}-{numeros[6:]}"
        if row['telefone1'] and (numeros := ''.join(filter(str.isdigit, str(row['telefone1']))))
        else (
            f"({(numeros := ''.join(filter(str.isdigit, str(row['telefone3']))))[:2]}) {numeros[2:6]}-{numeros[6:]}"
            if row['telefone3'] and (numeros := ''.join(filter(str.isdigit, str(row['telefone3']))))
            else None)),axis=1)

### Campo email
Qualquer um. Se houver mais de um vai para anotações gerais.

In [32]:
df['email'] = df.apply(
    lambda row: row['email1'] if row['email1'] else row['email2'], axis=1)
df['email_extra'] = df.apply(
    lambda row: row['email2'] if row['email1'] else None, axis=1)

### País
Na falta de informações específicas, como os clientes são brasileiros e os estados são brasileiros, será 'Brasil'

In [33]:
df['País'] = 'Brasil'

### Estado

Retornar apenas a sigla da Unidade Federativa

In [34]:
print(df['uf'].unique())

['SC' 'Sc' 'PR' None]


In [35]:
df['uf'] = df['uf'].str.upper()
print(df['uf'].unique())

['SC' 'PR' None]


In [36]:
df = df.rename(columns={'uf': 'Estado'})

### Cidade
Manter como está.
### Bairro
Manter como está.
### Endereço
Manter como está.
### CEP
Formatar em 8 dígitos. Colocar 0 na frente se necessário.

In [37]:
print(contar_algarismos_por_entrada(df, 'cep'))

8:500


Todas as entradas possuem apenas 8 dígitos.

In [38]:
df['cep'] = df['cep'].str.replace(r'\D', '', regex=True)

### PIS/PASEP
Instruções dizem para retornar valor na máscara 000.0000.000-0.

In [39]:
df['PIS'] = df['pis'].apply(lambda x: f"{str(x).zfill(11)[:3]}.{str(x).zfill(11)[3:7]}.{str(x).zfill(11)[7:10]}-{str(x).zfill(11)[10]}" if x else None)

### CTPS
Deverá ser mantida como está, porém essa informação não existe na tabela_clientes, logo permanecerá zerada.

### CID
Deverá ser mantida como está, porém essa informação não existe na tabela_clientes, logo permanecerá zerada.

### Nome da mãe
Manter como está.

### Origem do cliente
Não há informações sobre a origem na planilha, todos serão marcados como MIGRAÇÃO conforme instruções.

In [40]:
df['origemCliente'] = 'MIGRAÇÃO'

### Anotações gerais

Irá conter CPF e CNPJ, se inválidos, e e-mail extra, se houver.

In [41]:
df['infoGeral'] = df.apply(lambda row: 
    (f"CPF ou CNPJ do cliente é inválido: {row['CPFCNPJ_invalido']} | Email adicional do cliente: {row['email_extra']}" 
     if row['CPFCNPJ_invalido'] and row['email_extra'] 
     else f"CPF ou CNPJ do cliente é inválido: {row['CPFCNPJ_invalido']}" 
     if row['CPFCNPJ_invalido'] 
     else f"Email adicional do cliente: {row['email_extra']}" 
     if row['email_extra'] 
     else None), axis=1)

### Preparação da tabela nova

In [42]:
print(df_clientes.columns)

Index(['nomeCliente', 'cpfcnpj', 'rg', 'nacionalidade', 'dataDeNascimento',
       'estadoCivil', 'profissao', 'sexo', 'celular', 'telefone', 'email',
       'pais', 'estado', 'cidade', 'bairro', 'endereco', 'cep', 'pispasep',
       'ctps', 'cid', 'nomemae', 'origem', 'anotacoesGerais'],
      dtype='object')


In [43]:
df_clientes['nomeCliente'] = df['razao_social']
df_clientes['cpfcnpj'] = df['CPFCNPJ']
df_clientes['rg'] = df['rg']
df_clientes['nacionalidade'] = df['nacionalidade']
df_clientes['dataDeNascimento']=df['nascimento']
df_clientes['estadoCivil']=df['estado_civil']
df_clientes['profissao']=df['profissao']
df_clientes['celular']=df['celular']
df_clientes['telefone']=df['telefone']
df_clientes['email'] = df['email']
df_clientes['pais'] = 'Brasil'
df_clientes['estado'] = df['Estado']
df_clientes['cidade'] = df['cidade']
df_clientes['bairro'] = df['bairro']
df_clientes['endereco'] = df['logradouro']
df_clientes['cep'] = df['cep']
df_clientes['pispasep'] = df['PIS']
df_clientes['nomemae'] = df['nome_mae']
df_clientes['origem'] = 'MIGRAÇÃO'
df_clientes['anotacoesGerais'] = df['infoGeral']

In [44]:
df_clientes.replace({np.nan: None},inplace=True)
df_clientes.head()

,nomeCliente,cpfcnpj,rg,nacionalidade,dataDeNascimento,estadoCivil,profissao,sexo,celular,telefone,...,cidade,bairro,endereco,cep,pispasep,ctps,cid,nomemae,origem,anotacoesGerais
0,Fulano(a) 1,None,UF 1.234.567,Brasileira,27/02/1957,None,None,None,(00) 9999-1111,None,...,Floripa,Centro,"Rua dos Candidatos, Escola Advbox, n 1",12345600,None,None,None,Maria 1,MIGRAÇÃO,CPF ou CNPJ do cliente é inválido: 12345678900
1,Fulano(a) 2,None,12.123.123-0,Brasileira,13/01/1966,CA,None,None,(00) 9999-1112,None,...,Florianopolis,Centro,"Rua dos Candidatos, Escola Advbox, n 2",12345601,None,None,None,Maria 2,MIGRAÇÃO,CPF ou CNPJ do cliente é inválido: 12345678901
2,Fulano(a) 3,None,12.123.123-1,Brasileira,10/05/1982,CA,Empres√°rio,None,(00) 9999-1113,(00) 1234-4567,...,Florianopolis,Centro,"Rua dos Candidatos, Escola Advbox, n 3",12345602,123.4567.891-0,None,None,Maria 3,MIGRAÇÃO,CPF ou CNPJ do cliente é inválido: 12345678902...
3,Fulano(a) 4,None,12.123.123-2,Brasileira,01/01/1956,CA,Auxiliar de Servi√ßos Gerais,None,(00) 9999-1114,(00) 1234-4568,...,Florianopolis,Centro,"Rua dos Candidatos, Escola Advbox, n 4",12345603,123.4567.891-1,None,None,Maria 4,MIGRAÇÃO,CPF ou CNPJ do cliente é inválido: 12345678903
4,Fulano(a) 5,None,12.123.123-3,Brasileira,14/04/1963,CA,Industri√°rio,None,(00) 9999-1115,(00) 1234-4569,...,Floripa,Centro,"Rua dos Candidatos, Escola Advbox, n 5",12345604,123.4567.891-2,None,None,Maria 5,MIGRAÇÃO,CPF ou CNPJ do cliente é inválido: 12345678904


In [45]:
df_clientes = df_clientes.rename(columns={
    'nomeCliente': 'NOME',
    'cpfcnpj': 'CPF CNPJ',
    'rg': 'RG',
    'nacionalidade': 'NACIONALIDADE',
    'dataDeNascimento': 'DATA DE NASCIMENTO',
    'estadoCivil': 'ESTADO CIVIL',
    'profissao': 'PROFISSÃO',
    'sexo': 'SEXO',
    'celular': 'CELULAR',
    'telefone': 'TELEFONE',
    'email': 'EMAIL',
    'pais': 'PAÍS',
    'estado': 'ESTADO',
    'cidade': 'CIDADE',
    'bairro': 'BAIRRO',
    'endereco': 'ENDEREÇO',
    'cep': 'CEP',
    'pispasep': 'PIS PASEP',
    'ctps':'CTPS',
    'cid':'CID',
    'nomemae':'NOMEMAE',
    'origem':'ORIGEM DO CLIENTE',
    'anotacoesGerais':'ANOTAÇÕES GERAIS'
    })


In [46]:
df_clientes.to_excel('CLIENTES.xlsx', index=False)